In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

In [2]:
#prepare transform

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# prepare datasets

BATCH_SIZE = 50

train_datasets = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    transform=transform,
    download=True,
)

train_loader = torch.utils.data.DataLoader(
    train_datasets,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

In [4]:
test_datasets = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,  # ✅ テストデータにする
    transform=transform,
    download=True,
)

test_loader = torch.utils.data.DataLoader(
    test_datasets,
    batch_size=BATCH_SIZE,
    shuffle=False,  # テストなのでシャッフル不要
    num_workers=2
)

In [5]:
#make CNN model

class CNN(nn.Module):
    def __init__(self, use_forward1 = True):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # 修正: 64*8*8に変更
        self.fc2 = nn.Linear(128, 10)  # 10クラス分類
        
        self.use_forward1 = use_forward1
    
    def forward1(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # ✅ conv1 → ReLU → MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # ✅ conv2 → ReLU → MaxPool

        x = x.view(x.size(0), -1)  # ✅ Flatten（batch_size, 64 * 8 * 8）
        
        x = F.relu(self.fc1(x))  # ✅ FC1 → ReLU
        x = self.fc2(x)  # ✅ FC2（出力層）

        return x
    def forward2(self, x):
        x = F.relu(self.pool(self.conv1(x)))  # conv -> pool -> relu
        x = F.relu(self.pool(self.conv2(x)))
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

    def forward(self, x):
        
        if self.use_forward1:
            return self.forward1(x)
        else:
            return self.forward2(x)

In [6]:
model1 = CNN()
model2 = CNN()
model2.use_forward1 = False

criterion1 = nn.CrossEntropyLoss()  # 損失関数
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)  # 最適化手法

criterion2 = nn.CrossEntropyLoss()  # 損失関数
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)  # 最適化手法


In [7]:
#評価関数

def eval(model, test_loader,epoch, device="cpu"):
    model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    #writer.add_scalar("Accuracy/test", accuracy, epoch)
    return accuracy

In [8]:
#define training function
from torch.utils.tensorboard import SummaryWriter

def train(model, trainloader, criterion, optimizer, num_epochs = 5, device = "cpu"):
    model.to(device)
    #writer = SummaryWriter(log_dir="runs/cnn_exp_2025-02-28-epoch=10")
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total if total > 0 else 0

        """writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", epoch_acc, epoch)"""

        # 🔥 各エポックごとに `evaluate()` を呼び出す！
        test_acc = eval(model, test_loader, epoch, device)

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2%} - Test Acc: {test_acc:.2f}%")

    #writer.close()
    print("Training Finished")

In [9]:

train(model1, train_loader, criterion1, optimizer1)
train(model2, train_loader, criterion2, optimizer2)

Test Accuracy: 60.91%
Epoch 1/5 - Loss: 1.3162 - Train Acc: 52.96% - Test Acc: 60.91%
Test Accuracy: 68.01%
Epoch 2/5 - Loss: 0.9362 - Train Acc: 67.23% - Test Acc: 68.01%
Test Accuracy: 70.84%
Epoch 3/5 - Loss: 0.7910 - Train Acc: 72.29% - Test Acc: 70.84%
Test Accuracy: 71.43%
Epoch 4/5 - Loss: 0.6854 - Train Acc: 75.97% - Test Acc: 71.43%
Test Accuracy: 71.01%
Epoch 5/5 - Loss: 0.5875 - Train Acc: 79.22% - Test Acc: 71.01%
Training Finished
Test Accuracy: 63.02%
Epoch 1/5 - Loss: 1.3105 - Train Acc: 53.24% - Test Acc: 63.02%
Test Accuracy: 68.05%
Epoch 2/5 - Loss: 0.9371 - Train Acc: 67.06% - Test Acc: 68.05%
Test Accuracy: 70.36%
Epoch 3/5 - Loss: 0.7822 - Train Acc: 72.60% - Test Acc: 70.36%
Test Accuracy: 72.07%
Epoch 4/5 - Loss: 0.6648 - Train Acc: 76.66% - Test Acc: 72.07%
Test Accuracy: 72.39%
Epoch 5/5 - Loss: 0.5578 - Train Acc: 80.40% - Test Acc: 72.39%
Training Finished
